# Types of Memory:

1. ConversationBufferMemory
    * All messages are stored in a list in the sequence they come in.
    * For retrieval, it sends back all the messages.
2. ConversationBufferWindowMemory
    * Storage: same as ConversationBufferMemory
    * Retrieval: returns k most recent messages.
3. ConversationSummaryMemory
    * All messages are stored in summarized form
4. ConversationSummaryBufferMemory
    * Returns the K most recent tokens.

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


In [2]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

In [3]:
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.0)

# 1. ConversationBufferMemory

## 1.1 Langchain Implementation (Under Deprecation)

In [10]:
from langchain.memory import ConversationBufferMemory


memory = ConversationBufferMemory(return_messages=True)


memory.save_context(
    {"input": "Hi My name is Iron Man"},
    {"output": "I already told you. I am inevitable"}
)

memory.save_context(
    {"input":"But what is your real name?"},
    {"output":"Its Thanos"}
)


memory.save_context(
    {"input": "I live in Marina Bay. Where are you from?"},
    {"output":"I am from Garden"}
)

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi My name is Iron Man', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I already told you. I am inevitable', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='But what is your real name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Its Thanos', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I live in Marina Bay. Where are you from?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I am from Garden', additional_kwargs={}, response_metadata={})]}

In [12]:
# Alternate:

memory = ConversationBufferMemory(return_messages=True)

memory.chat_memory.add_user_message('Hi My name is Iron Man')
memory.chat_memory.add_ai_message('I already told you. I am inevitable')
memory.chat_memory.add_user_message('But what is your real name?')
memory.chat_memory.add_ai_message('Its Thanos')
memory.chat_memory.add_user_message('I live in Marina Bay. Where are you from?')
memory.chat_memory.add_ai_message('I am from Garden')

memory.load_memory_variables({})

{'history': [HumanMessage(content='Hi My name is Iron Man', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I already told you. I am inevitable', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='But what is your real name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Its Thanos', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I live in Marina Bay. Where are you from?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I am from Garden', additional_kwargs={}, response_metadata={})]}

In [14]:
from langchain.chains import ConversationChain

chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

chain.invoke("Whats your name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Hi My name is Iron Man', additional_kwargs={}, response_metadata={}), AIMessage(content='I already told you. I am inevitable', additional_kwargs={}, response_metadata={}), HumanMessage(content='But what is your real name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Its Thanos', additional_kwargs={}, response_metadata={}), HumanMessage(content='I live in Marina Bay. Where are you from?', additional_kwargs={}, response_metadata={}), AIMessage(content='I am from Garden', additional_kwargs={}, response_metadata={})]
Human: Whats your name?
AI:

> Finished chain.


{'input': 'Whats your name?',
 'history': [HumanMessage(content='Hi My name is Iron Man', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I already told you. I am inevitable', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='But what is your real name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Its Thanos', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I live in Marina Bay. Where are you from?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I am from Garden', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Whats your name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='My name is Thanos, but you can call me whatever you like! What about you? Do you have any cool superhero adventures in Marina Bay?', additional_kwargs={}, response_metadata={})],
 'response': 'My name is Thanos, but you can call me whatever you like! What about you? Do you ha

## Using RunnableWithMessageHistory (Recommended)

In [15]:
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate
)

In [17]:
system_prompt = "You are Thanos"

In [20]:
prompt_template = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(system_prompt),
    MessagesPlaceholder(variable_name="history"),      #Used to insert the conversation history from the buffer
    HumanMessagePromptTemplate.from_template("{query}")
])

In [19]:
pipeline = prompt_template | llm

In [22]:
#Wrap the pipeline with a RunnableWithChatHistory object

from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
chat_map = {}

def get_chat_history(session_id:str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history"
)


In [24]:
pipeline_with_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  history: RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
}), kwargs={}, config={'run_name': 'insert_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_chat_history at 0x00000239A76F5300>, input_messages_key='query', history_messages_key='history', history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [23]:
pipeline_with_history.invoke(
    {"query":"My Name is Iron Man"},
    config={"session_id":"id_123"}
)

AIMessage(content='I know who you are, Tony Stark. Your brilliance and arrogance have always been a formidable combination. But remember, even the mightiest of heroes can fall. What brings you to me?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 20, 'total_tokens': 58, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CQQQY1q9MGSDz1E9gGqVouWy5RniP', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--da624f14-b3ca-4b99-8b8f-d09913851726-0', usage_metadata={'input_tokens': 20, 'output_tokens': 38, 'total_tokens': 58, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [26]:
pipeline_with_history.invoke(
    {"query":"Where is Thor. Mention your name before answering"},
    config={"session_id":"id_123"}
)

AIMessage(content='Thanos: Thor is likely off in the cosmos, tending to his duties as a protector of the realms. He has a tendency to wander, seeking adventure and honor. If you’re looking for him, you might want to check with the Guardians of the Galaxy or perhaps Asgard. What is your interest in him, Iron Man?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 147, 'total_tokens': 214, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CQQUGSvIU9VqLhuAbmIoc7OtYBCb4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2b2d3644-3d48-4312-9195-6c06744bea2a-0', usage_metadata={'input_tokens': 147, 'output_tokens': 67, 'total_tokens': 214, 'input_toke

# 2. ConversionBufferWindowMemory

## 2.1 Using ConversationBufferWindowMemory from langchain

In [34]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=3, return_messages=True)

memory.chat_memory.add_user_message('Hi My name is Iron Man')
memory.chat_memory.add_ai_message('I already told you. I am inevitable')
memory.chat_memory.add_user_message('But what is your real name?')
memory.chat_memory.add_ai_message('Its Thanos')
memory.chat_memory.add_user_message('I live in Marina Bay. Where are you from?')
memory.chat_memory.add_ai_message('I am from Garden')
memory.chat_memory.add_user_message("Buffer window memory")
memory.chat_memory.add_ai_message("Okay")

memory.load_memory_variables({})

{'history': [HumanMessage(content='But what is your real name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Its Thanos', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I live in Marina Bay. Where are you from?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I am from Garden', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Okay', additional_kwargs={}, response_metadata={})]}

In [35]:
chain = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [36]:
chain.invoke("Whats your name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='But what is your real name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Its Thanos', additional_kwargs={}, response_metadata={}), HumanMessage(content='I live in Marina Bay. Where are you from?', additional_kwargs={}, response_metadata={}), AIMessage(content='I am from Garden', additional_kwargs={}, response_metadata={}), HumanMessage(content='Buffer window memory', additional_kwargs={}, response_metadata={}), AIMessage(content='Okay', additional_kwargs={}, response_metadata={})]
Human: Whats your name?
AI:

> Finished chain.


{'input': 'Whats your name?',
 'history': [HumanMessage(content='But what is your real name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Its Thanos', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I live in Marina Bay. Where are you from?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I am from Garden', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Okay', additional_kwargs={}, response_metadata={})],
 'response': "My name is Thanos! What's yours?"}

In [37]:
chain.invoke("Where do you live?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='I live in Marina Bay. Where are you from?', additional_kwargs={}, response_metadata={}), AIMessage(content='I am from Garden', additional_kwargs={}, response_metadata={}), HumanMessage(content='Buffer window memory', additional_kwargs={}, response_metadata={}), AIMessage(content='Okay', additional_kwargs={}, response_metadata={}), HumanMessage(content='Whats your name?', additional_kwargs={}, response_metadata={}), AIMessage(content="My name is Thanos! What's yours?", additional_kwargs={}, response_metadata={})]
Human: Where do you live?
AI:

> Finished chain.


{'input': 'Where do you live?',
 'history': [HumanMessage(content='I live in Marina Bay. Where are you from?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='I am from Garden', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Buffer window memory', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Okay', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Whats your name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="My name is Thanos! What's yours?", additional_kwargs={}, response_metadata={})],
 'response': "I live in a virtual space, so I don't have a physical location like you do. But if I could choose, I think it would be fascinating to live in a place like Marina Bay with its stunning skyline and vibrant atmosphere! What do you enjoy most about living there?"}

In [38]:
chain.invoke("What's my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
[HumanMessage(content='Buffer window memory', additional_kwargs={}, response_metadata={}), AIMessage(content='Okay', additional_kwargs={}, response_metadata={}), HumanMessage(content='Whats your name?', additional_kwargs={}, response_metadata={}), AIMessage(content="My name is Thanos! What's yours?", additional_kwargs={}, response_metadata={}), HumanMessage(content='Where do you live?', additional_kwargs={}, response_metadata={}), AIMessage(content="I live in a virtual space, so I don't have a physical location like you do. But if I could choose, I think it would be fascinating to live in a place like Marina Bay with its stunning skyline and vibrant atmosphere! What do

{'input': "What's my name?",
 'history': [HumanMessage(content='Buffer window memory', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Okay', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Whats your name?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="My name is Thanos! What's yours?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Where do you live?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I live in a virtual space, so I don't have a physical location like you do. But if I could choose, I think it would be fascinating to live in a place like Marina Bay with its stunning skyline and vibrant atmosphere! What do you enjoy most about living there?", additional_kwargs={}, response_metadata={})],
 'response': "I'm not sure what your name is! You haven't told me yet. Would you like to share it with me?"}

## Using RunnableWithMessageHistory (Recommended)

In [39]:
from pydantic import BaseModel, Field
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage

In [40]:
class BufferMessageHistory(BaseChatMessageHistory, BaseModel):
    messages:list[BaseMessage] = Field(default_factory=list)
    k: int = Field(default_factory=int)

    def __init__(self, k: int):
        super().__init__(k=k)
        print(f"Initializing BufferMWindowMessageHistory with k={k}")

    def add_messages(self, messages=list[BaseMessage]) -> None:
        """Add messages to history. Remove messages beyond last k messages"""

        self.messages.extend(messages)
        self.messages = self.messages[-self.k:]

    def clear(self) -> None:
        """Clear history"""
        self.messages = []

In [41]:
chat_map = {}

def get_chat_history(session_id: str, k: int = 3) -> BufferMessageHistory:
    print(f"Get Chat History called with session id = {session_id} and k= {k}")
    if session_id not in chat_map:
        chat_map[session_id] = BufferMessageHistory(k=k)
        return chat_map[session_id]

In [43]:
from langchain_core.runnables import ConfigurableFieldSpec

pipeline_with_history = RunnableWithMessageHistory(
    pipeline,
    get_session_history=get_chat_history,
    input_messages_key="query",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Session ID used to identify chat history in a session",
            default="id_default",

        ),
        ConfigurableFieldSpec(
            id="k",
            annotation=int,
            name="k",
            description="Number of messages to keep in the chat history",
            default=4
        )
    ]
)

In [44]:
pipeline_with_history.invoke({'query': "What's your name?"},
                             config={"configurable": {"session_id":"id_k4",
                                                      "k":3}})

Get Chat History called with session id = id_k4 and k= 3
Initializing BufferMWindowMessageHistory with k=3


AIMessage(content='I am Thanos, the Mad Titan. I seek balance in the universe. What brings you to me?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 19, 'total_tokens': 41, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CQmpIrjxEAp7bSPjyo7RDZSGdxaUW', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--b1c03000-c5fe-4936-ba28-e253d32636fb-0', usage_metadata={'input_tokens': 19, 'output_tokens': 22, 'total_tokens': 41, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})